In [ ]:
# ・今後の方針
# ・HUGにしていくため、まず２次元でデータの読み込みを行なっていく
# ・体育館のデータを作成する。静岡県のものを参考にする。
# ・まずは校舎全体ではなく、体育館だけの図面で検証する。
# ・検証内容としては、【体育館の図面をNPCが把握した発言をしていること】
# ・
# 
# 
# 
# .


In [ ]:
import random
import time
import re
import os
import json
import pandas as pd
from openai import OpenAI
from dotenv import load_dotenv

# --- 環境変数の読み込み ---
load_dotenv()
client = OpenAI()

# --- NPCプロファイルの読み込み（1人分） ---
with open("npc_profiles.json", encoding="utf-8") as f:
    npc_list = json.load(f)
    single_npc = npc_list[0]

# --- 備考から状況（全壊など）を抽出する関数 ---
def extract_situation(text):
    keywords = ["全壊", "半壊", "一部損壊", "全焼", "無被害", "被害なし"]
    for word in keywords:
        if word in text:
            return word
    return "不明"

# --- 避難者本人の自己紹介文を生成 ---
def evacuee_profile_to_statement(evacuee):
    return f"{evacuee['名前']}です。{evacuee['構成']}で避難してきました。状況は{evacuee['状況']}です。できれば家族で安心できる場所で過ごしたいです。"

# --- CSVから避難者データ読み込み ---
df = pd.read_csv("hinanzyo_events.csv", encoding="utf-8", dtype=str)
evacuees_df = df[df["タイプ"] == "避難者"].dropna(subset=["名前", "地区", "備考"])

evacuees = []
for _, row in evacuees_df.iterrows():
    備考 = str(row["備考"]).strip()
    evacuees.append({
        "名前": str(row["名前"]).strip(),
        "構成": 備考,
        "状況": extract_situation(備考),
        "地区": str(row["地区"]).strip()
    })

# --- 体育館のグリッド定義 ---
GRID_WIDTH, GRID_HEIGHT = 20, 15
gym_grid = pd.DataFrame(" ", index=range(GRID_HEIGHT), columns=range(GRID_WIDTH))

area_types = {
    "器": [(14, 0), (17, 0)],
    "ス": [(14, 1), (17, 3)],
    "出": [(0, 0), (2, 2)],
    "体": [(3, 3), (13, 9)]
}
for label, ((x1, y1), (x2, y2)) in area_types.items():
    for y in range(y1, y2 + 1):
        for x in range(x1, x2 + 1):
            gym_grid.at[y, x] = label

# --- グリッド文字列化 ---
def stringify_gym_grid(gym_df):
    text = ""
    for y in range(gym_df.shape[0]):
        row = []
        for x in range(gym_df.shape[1]):
            value = gym_df.at[y, x]
            row.append(f"({x},{y})={value if value != ' ' else '空'}")
        text += " ".join(row) + "\n"
    return text

# --- NPCの発言生成 ---
def generate_npc_response(npc, evacuee, gym_text, conversation_log):
    evacuee_info = f"名前：{evacuee['名前']}\n構成：{evacuee['構成']}\n状況：{evacuee['状況']}\n地区：{evacuee['地区']}"
    prompt = f"""あなたは避難所運営に関わるNPC「{npc['name']}」です。
役割：{npc['role']}、性格：{npc['personality']}、話し方：{npc['style']}。

【避難者情報】
{evacuee_info}

【体育館グリッド状況】
{gym_text}

この会話では、この避難者の情報と体育館のグリッド状況をもとに、どこに配置すべきかについて議論します。行政職員の剛史君（リーダー）の発言を参考に、次のいずれかの形で応答してください：

1. 質問：リーダーの発言を深掘りする質問をする  
2. 賛成 or 反対：リーダーの発言に立場を明確に示し、理由を述べる  
3. 課題提起：新たな観点やリスクを提案する（議論を広げる）

※ ファシリテーターとして話を整理したり、議論の方向性を提示しても構いません。
以下の会話ログを参考に、一文で返答してください。
"""
    log_text = "\n".join([f"{entry['speaker']}：{entry['content']}" for entry in conversation_log])
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": prompt},
                {"role": "user", "content": log_text}
            ],
            temperature=0.7
        )
        content = response.choices[0].message.content.strip()
        content = re.sub(rf"^{npc['name']}[:：]\s*", "", content)
        return content.strip()
    except Exception as e:
        return f"（エラー：{e}）"

# --- 会話開始 ---
print("【剛史君の役割】")
print("あなたは避難所担当の行政職員であり、避難所運営の議論をリードする立場にあります。")
print("この会話の目的は、様々な立場の人の意見を聞き、避難所としての方針を決定することです。\n")

num_turns = 1  # NPCが1回話す

for evacuee in evacuees:
    print(f"\n【イベント】避難者「{evacuee['名前']}」さんが来ました（地区：{evacuee['地区']}）\n")

    conversation_log = [{"speaker": "system", "content": f"避難者『{evacuee['名前']}』来訪イベント"}]

    # 避難者本人の自己紹介
    evacuee_speech = evacuee_profile_to_statement(evacuee)
    conversation_log.append({"speaker": evacuee["名前"], "content": evacuee_speech})
    print(f"{evacuee['名前']}：{evacuee_speech}")

    # 剛史君の応答
    first_input = input("剛史君、避難者の話を受けて発言してください：").strip()
    if first_input:
        msg = {"speaker": "剛史君", "content": first_input}
        conversation_log.append(msg)
        print(f"{msg['speaker']}：{msg['content']}")
    else:
        print("（剛史君からの発言がありません）")


    # グリッド情報作成
    grid_text = stringify_gym_grid(gym_grid)

    # NPC応答
    for _ in range(num_turns):
        response = generate_npc_response(single_npc, evacuee, grid_text, conversation_log)
        conversation_log.append({"speaker": single_npc["name"], "content": response})
        print(f"{single_npc['name']}：{response}")
        time.sleep(1.2)

    # 剛史君の最終判断
    decision = input("剛史君、皆の意見を踏まえて配置方針を述べてください：").strip()
    if decision:
        conversation_log.append({"speaker": "剛史君", "content": decision})
        print(f"剛史君：{decision}")


【剛史君の役割】
あなたは避難所担当の行政職員であり、避難所運営の議論をリードする立場にあります。
この会話の目的は、様々な立場の人の意見を聞き、避難所としての方針を決定することです。


【イベント】避難者「床上さん」さんが来ました（地区：西浦）

床上さん：床上さんです。３人（本人・妻・長男）・全壊で避難してきました。状況は全壊です。できれば家族で安心できる場所で過ごしたいです。
（剛史君からの発言がありません）
佐藤さん：佐藤です。床上さん、ご家族が全壊の状況で避難されてきたとのこと、大変お疲れ様です。家族で安心できる場所を確保するためには、どのグリッドが最も適しているかを考慮する必要がありますが、リーダーの剛史君、どのような配置を検討されますか？
